<a href="https://colab.research.google.com/github/mazarimono/members_handson/blob/master/members_handson_hogawa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Members. data #2 データ可視化・ライブラリLT編

## Dashを使ってデータ分析

### ハンズオン的解説





## 自己紹介
- hideyuki ogawa
- 合同会社長目　経営
- はんなりPython 運営
- 証券アナリスト資格保有

## イベント
- はんなりPython データ分析部
  - ワイワイデータ分析を行う会　
  - https://hannari-python.connpass.com/event/184468/
- PyConJP Tutorial 「オープンデータを使って、Pythonでデータ分析」
  - データの取得、前処理、可視化、機械学習を行うチュートリアル
  - https://pyconjp.connpass.com/event/181065/

## 今日やること
- データを取り込み
- データを前処理
- データを可視化
- データをインタラクティブに可視化
- データをよりインタラクティブに可視化

In [1]:
!pip install dash jupyter_dash requests_html
!pip install -U plotly

     |████████████████████████████████| 71kB 4.3MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 1.2MB 13.7MB/s 
     |████████████████████████████████| 3.4MB 33.4MB/s 
     |████████████████████████████████| 194kB 46.8MB/s 
     |████████████████████████████████| 1.8MB 43.9MB/s 
     |████████████████████████████████| 153kB 41.3MB/s 
     |████████████████████████████████| 358kB 40.2MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
  Created wheel for dash: filename=dash-1.14.0-cp36-none-any.whl size=80129 sha256=411751c523867448fc2c1175f29e42131ed003ab2b511739e6c009fc55981826
  Stored in directory: /root/.cache/pip/wheels/1d/54/0b/e846f762d47e8abc9234419822d013dfb2ccf957b48e411dc6
  Created wheel for flask-compress: filename=Flask_Compress-1.5.0-cp36-none-any.whl size=5273 sha256=34eb11ef6de704186c19fadbafa05f4bcd1e500f7596993521c921c14c057136
  Stored in directory: /roo

     |████████████████████████████████| 12.9MB 305kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [31]:
import base64
from io import StringIO, BytesIO
from jupyter_dash import JupyterDash
import dash_core_components as dcc 
import dash_html_components as html 
from dash_table import DataTable
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
from requests_html import HTMLSession

In [ ]:
from google.colab import drive  
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!mkdir data

## まずは本日使うデータを読み込みます。
- 利用するのは家計調査の支出金額（出典：総務省統計局のページ https://www.stat.go.jp/data/kakei/longtime/index.html#time)
- コロナ後の消費動向の変化を見ながら、Dashに関して解説していきます

In [3]:
# 普通はこのような感じで読み込める

df = pd.read_csv("https://www.stat.go.jp/data/kakei/longtime/csv/h-mon-a.csv")


In [4]:
# しかし何らかの力で直接読み込めなくなっている
df

,<html><head><title>Request Rejected</title></head><body>The requested URL was rejected. Please consult with your administrator.<br><br>Your support ID is: 6133672767692208960</body></html>


In [5]:
# そのため、ちょっと工夫を
session = HTMLSession()
r = session.get("https://www.stat.go.jp/data/kakei/longtime/csv/h-mon-a.csv")
r.text[:100]

'\x93ñ\x90l\x88È\x8fã\x82Ì\x90¢\x91Ñ_\x8ex\x8fo\x8bà\x8az[\x89~],,,,,,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,2'

In [6]:
# 読み込めたが文字化けしているので、エンコーディングを更新し（"shift-jis"）、ファイル化しread_csvを使って読み込む
r.encoding = "shift-jis"
data = StringIO(r.text)

df = pd.read_csv(data)

In [7]:
df.head(20)

,二人以上の世帯_支出金額[円],Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,...,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245
0,NaN,NaN,NaN,NaN,NaN,NaN,2000年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019年,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020年,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,(平成12年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(平成13年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(平成14年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(平成30年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(平成31年),NaN,NaN,NaN,(令和元年),NaN,NaN,NaN,NaN,NaN,NaN,NaN,(令和2年),NaN,NaN,NaN,NaN
2,表側連番,階層,大分類,中分類,小分類,品目分類,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,...,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月,6月,7月,8月,9月,10月,11月,12月,1月,2月,3月,4月,5月
3,1,-,-,-,-,世帯数分布(抽出率調整),10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
4,2,-,-,-,-,集計世帯数,7887,7942,7934,7922,7928,7917,7907,7908,7917,7937,7914,7933,7888,7887,7907,7927,7948,7924,7907,7909,7910,7908,7914,7924,7857,7877,7915,7894,7932,7926,7914,7893,7914,7933,...,7714,7689,7719,7735,7706,7710,7738,7729,7737,7678,7671,7596,7654,7656,7662,7648,7649,7648,7671,7617,7620,7600,7639,7555,7572,7581,7541,7548,7553,7501,7479,7517,7470,7466,7481,7443,7475,7489,7534,7430
5,3,-,-,-,-,世帯人員(人),3.32,3.32,3.32,3.32,3.31,3.31,3.31,3.31,3.31,3.3,3.3,3.29,3.29,3.29,3.29,3.29,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.26,3.25,3.25,3.25,3.24,3.24,3.24,3.24,3.24,3.24,...,2.98,2.98,2.97,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.97,2.97,2.97,2.97,2.97,2.98,2.98,2.97,2.98,2.98,2.98,2.98,2.98,2.97,2.97,2.97,2.97,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96
6,4,-,-,-,-,18歳未満人員(人),0.74,0.75,0.75,0.75,0.75,0.74,0.74,0.74,0.74,0.73,0.72,0.73,0.72,0.72,0.72,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.7,0.69,0.7,0.69,0.69,0.7,0.7,0.7,0.7,0.71,0.71,...,0.54,0.54,0.54,0.55,0.55,0.55,0.54,0.55,0.55,0.55,0.55,0.55,0.55,0.55,0.54,0.55,0.55,0.56,0.56,0.56,0.57,0.58,0.59,0.58,0.59,0.59,0.59,0.58,0.57,0.57,0.57,0.58,0.58,0.58,0.58,0.58,0.58,0.57,0.58,0.58
7,5,-,-,-,-,65歳以上人員(人),0.52,0.53,0.53,0.52,0.52,0.53,0.54,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.57,0.58,0.57,0.56,0.57,0.57,0.57,0.56,0.55,0.57,0.55,0.57,0.58,0.58,0.58,0.58,0.58,0.59,0.58,0.58,...,0.85,0.86,0.87,0.86,0.86,0.86,0.86,0.85,0.85,0.85,0.85,0.84,0.84,0.84,0.84,0.84,0.84,0.83,0.82,0.83,0.83,0.82,0.82,0.81,0.82,0.83,0.83,0.83,0.83,0.84,0.83,0.82,0.82,0.82,0.82,0.82,0.83,0.86,0.86,0.85
8,6,-,-,-,-,うち無職者人員(人),0.41,0.41,0.41,0.41,0.41,0.42,0.42,0.42,0.42,0.42,0.43,0.42,0.43,0.44,0.44,0.45,0.44,0.44,0.44,0.44,0.45,0.44,0.44,0.45,0.43,0.45,0.45,0.46,0.46,0.47,0.47,0.47,0.47,0.47,...,0.67,0.68,0.68,0.68,0.69,0.7,0.69,0.67,0.67,0.67,0.67,0.66,0.67,0.67,0.66,0.67,0.67,0.65,0.65,0.65,0.65,0.64,0.65,0.64,0.65,0.66,0.66,0.66,0.66,0.67,0.67,0.66,0.66,0.66,0.66,0.65,0.66,0.68,0.68,0.68
9,7,-,-,-,-,有業人員(人),1.51,1.51,1.51,1.52,1.53,1.53,1.52,1.52,1.51,1.52,1.53,1.52,1.51,1.51,1.51,1.52,1.5,1.53,1.52,1.51,1.51,1.49,1.49,1.48,1.48,1.45,1.45,1.46,1.44,1.45,1.44,1.44,1.44,1.44,...,1.32,1.32,1.32,1.32,1.32,1.3,1.32,1.32,1.33,1.34,1.34,1.34,1.34,1.34,1.36,1.35

## データを使いやすくするため前処理する
- 現状いろいろデータがあるが、各商品のデータを見るツールを作成するためのデータ作り
- 品目分類のデータを時系列で持つものを作成する

In [8]:
# データを前処理する

year_list = df.iloc[0, :].dropna().values
latest_month = int(df.iloc[2, -1].replace("月", ""))

month_count = (len(year_list)- 1) * 12 + latest_month

In [9]:
# pandasのdate_range関数を使って日付を作成

date_col = pd.date_range(start="2000", periods=month_count, freq="M")
date_col

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-30', '2000-10-31',
               ...
               '2019-08-31', '2019-09-30', '2019-10-31', '2019-11-30',
               '2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31',
               '2020-04-30', '2020-05-31'],
              dtype='datetime64[ns]', length=245, freq='M')

In [10]:
# 使う部分のデータを選択する
df.iloc[3:, 5:]

,Unnamed: 5,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245
3,世帯数分布(抽出率調整),10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
4,集計世帯数,7887,7942,7934,7922,7928,7917,7907,7908,7917,7937,7914,7933,7888,7887,7907,7927,7948,7924,7907,7909,7910,7908,7914,7924,7857,7877,7915,7894,7932,7926,7914,7893,7914,7933,7905,7932,7850,7879,7917,...,7714,7689,7719,7735,7706,7710,7738,7729,7737,7678,7671,7596,7654,7656,7662,7648,7649,7648,7671,7617,7620,7600,7639,7555,7572,7581,7541,7548,7553,7501,7479,7517,7470,7466,7481,7443,7475,7489,7534,7430
5,世帯人員(人),3.32,3.32,3.32,3.32,3.31,3.31,3.31,3.31,3.31,3.3,3.3,3.29,3.29,3.29,3.29,3.29,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.26,3.25,3.25,3.25,3.24,3.24,3.24,3.24,3.24,3.24,3.24,3.23,3.24,3.23,3.23,...,2.98,2.98,2.97,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.97,2.97,2.97,2.97,2.97,2.98,2.98,2.97,2.98,2.98,2.98,2.98,2.98,2.97,2.97,2.97,2.97,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96
6,18歳未満人員(人),0.74,0.75,0.75,0.75,0.75,0.74,0.74,0.74,0.74,0.73,0.72,0.73,0.72,0.72,0.72,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.7,0.69,0.7,0.69,0.69,0.7,0.7,0.7,0.7,0.71,0.71,0.7,0.69,0.69,0.69,0.68,...,0.54,0.54,0.54,0.55,0.55,0.55,0.54,0.55,0.55,0.55,0.55,0.55,0.55,0.55,0.54,0.55,0.55,0.56,0.56,0.56,0.57,0.58,0.59,0.58,0.59,0.59,0.59,0.58,0.57,0.57,0.57,0.58,0.58,0.58,0.58,0.58,0.58,0.57,0.58,0.58
7,65歳以上人員(人),0.52,0.53,0.53,0.52,0.52,0.53,0.54,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.57,0.58,0.57,0.56,0.57,0.57,0.57,0.56,0.55,0.57,0.55,0.57,0.58,0.58,0.58,0.58,0.58,0.59,0.58,0.58,0.58,0.58,0.58,0.59,0.59,...,0.85,0.86,0.87,0.86,0.86,0.86,0.86,0.85,0.85,0.85,0.85,0.84,0.84,0.84,0.84,0.84,0.84,0.83,0.82,0.83,0.83,0.82,0.82,0.81,0.82,0.83,0.83,0.83,0.83,0.84,0.83,0.82,0.82,0.82,0.82,0.82,0.83,0.86,0.86,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,会費・つきあい費,1561,1441,1495,1888,1683,1585,1566,1390,1297,1390,1339,1603,1366,1374,1432,1735,1562,1519,1401,1269,1334,1241,1259,1467,1254,1324,1511,1778,1532,1453,1393,1161,1233,1252,1242,1508,1254,1144,1442,...,693,806,1007,740,807,751,716,706,713,784,837,737,714,786,948,791,859,788,735,785,762,820,968,832,813,831,1063,878,929,864,707,802,710,776,860,770,705,690,751,696
184,その他の教養娯楽,7063,6370,8068,8109,8822,6858,7033,7995,6681,6831,7450,13030,7045,6272,7659,8213,9111,6615,6897,7755,6878,6794,7695,11965,7404,6274,8189,8577,9195,7119,7373,8510,7172,7335,7753,12380,7045,6670,7959,...,7001,8999,9292,9880,8448,8278,9543,8197,7978,8806,12271,8103,7239,9220,8933,9453,7860,8634,9431,8375,8474,8814,11937,8646,7965,8659,9426,10263,8505,8641,9456,9369,8284,9152,12570,8239,7674,8995,8603,9612
185,消費支出(再掲),309621,290663,335341,335276,308566,297648,326480,309993,296457,309193,297144,391556,308912,293698,339910,318509,300346,286054,314251,302474,283973,305653,294929,359940,305221,273487,330380,320228,291968,289427,315699,298788,295423,304989,289126,356702,298734,270619,321847,...,260644,297942,295929,283056,268802,279197,280320,268802,282872,277361,322157,289703,265614,301230,294439,281307,267641,283387,292481,271273,290396,281041,329271,296345,271232,309274

In [12]:
df1 = df.iloc[3:, 5:]
df1 = df1.set_index("Unnamed: 5")
df1

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245
Unnamed: 5,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
世帯数分布(抽出率調整),10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
集計世帯数,7887,7942,7934,7922,7928,7917,7907,7908,7917,7937,7914,7933,7888,7887,7907,7927,7948,7924,7907,7909,7910,7908,7914,7924,7857,7877,7915,7894,7932,7926,7914,7893,7914,7933,7905,7932,7850,7879,7917,7911,...,7714,7689,7719,7735,7706,7710,7738,7729,7737,7678,7671,7596,7654,7656,7662,7648,7649,7648,7671,7617,7620,7600,7639,7555,7572,7581,7541,7548,7553,7501,7479,7517,7470,7466,7481,7443,7475,7489,7534,7430
世帯人員(人),3.32,3.32,3.32,3.32,3.31,3.31,3.31,3.31,3.31,3.3,3.3,3.29,3.29,3.29,3.29,3.29,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.26,3.25,3.25,3.25,3.24,3.24,3.24,3.24,3.24,3.24,3.24,3.23,3.24,3.23,3.23,3.23,...,2.98,2.98,2.97,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.97,2.97,2.97,2.97,2.97,2.98,2.98,2.97,2.98,2.98,2.98,2.98,2.98,2.97,2.97,2.97,2.97,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96
18歳未満人員(人),0.74,0.75,0.75,0.75,0.75,0.74,0.74,0.74,0.74,0.73,0.72,0.73,0.72,0.72,0.72,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.7,0.69,0.7,0.69,0.69,0.7,0.7,0.7,0.7,0.71,0.71,0.7,0.69,0.69,0.69,0.68,0.67,...,0.54,0.54,0.54,0.55,0.55,0.55,0.54,0.55,0.55,0.55,0.55,0.55,0.55,0.55,0.54,0.55,0.55,0.56,0.56,0.56,0.57,0.58,0.59,0.58,0.59,0.59,0.59,0.58,0.57,0.57,0.57,0.58,0.58,0.58,0.58,0.58,0.58,0.57,0.58,0.58
65歳以上人員(人),0.52,0.53,0.53,0.52,0.52,0.53,0.54,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.57,0.58,0.57,0.56,0.57,0.57,0.57,0.56,0.55,0.57,0.55,0.57,0.58,0.58,0.58,0.58,0.58,0.59,0.58,0.58,0.58,0.58,0.58,0.59,0.59,0.59,...,0.85,0.86,0.87,0.86,0.86,0.86,0.86,0.85,0.85,0.85,0.85,0.84,0.84,0.84,0.84,0.84,0.84,0.83,0.82,0.83,0.83,0.82,0.82,0.81,0.82,0.83,0.83,0.83,0.83,0.84,0.83,0.82,0.82,0.82,0.82,0.82,0.83,0.86,0.86,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
会費・つきあい費,1561,1441,1495,1888,1683,1585,1566,1390,1297,1390,1339,1603,1366,1374,1432,1735,1562,1519,1401,1269,1334,1241,1259,1467,1254,1324,1511,1778,1532,1453,1393,1161,1233,1252,1242,1508,1254,1144,1442,1521,...,693,806,1007,740,807,751,716,706,713,784,837,737,714,786,948,791,859,788,735,785,762,820,968,832,813,831,1063,878,929,864,707,802,710,776,860,770,705,690,751,696
その他の教養娯楽,7063,6370,8068,8109,8822,6858,7033,7995,6681,6831,7450,13030,7045,6272,7659,8213,9111,6615,6897,7755,6878,6794,7695,11965,7404,6274,8189,8577,9195,7119,7373,8510,7172,7335,7753,12380,7045,6670,7959,8229,...,7001,8999,9292,9880,8448,8278,9543,8197,7978,8806,12271,8103,7239,9220,8933,9453,7860,8634,9431,8375,8474,8814,11937,8646,7965,8659,9426,10263,8505,8641,9456,9369,8284,9152,12570,8239,7674,8995,8603,9612
消費支出(再掲),309621,290663,335341,335276,308566,297648,326480,309993,296457,309193,297144,391556,308912,293698,339910,318509,300346,286054,314251,302474,283973,305653,294929,359940,305221,273487,330380,320228,291968,289427,315699,298788,295423,304989,289126,356702,298734,270619,321847,315074,...,260644,297942,295929,283056,268802,279197,280320,268802,282872,277361,322157

In [13]:
df1.index.names = [""]
df1.columns = date_col
df1.head()

,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,2000-09-30,2000-10-31,2000-11-30,2000-12-31,2001-01-31,2001-02-28,2001-03-31,2001-04-30,2001-05-31,2001-06-30,2001-07-31,2001-08-31,2001-09-30,2001-10-31,2001-11-30,2001-12-31,2002-01-31,2002-02-28,2002-03-31,2002-04-30,2002-05-31,2002-06-30,2002-07-31,2002-08-31,2002-09-30,2002-10-31,2002-11-30,2002-12-31,2003-01-31,2003-02-28,2003-03-31,2003-04-30,...,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
世帯数分布(抽出率調整),10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,...,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
集計世帯数,7887,7942,7934,7922,7928,7917,7907,7908,7917,7937,7914,7933,7888,7887,7907,7927,7948,7924,7907,7909,7910,7908,7914,7924,7857,7877,7915,7894,7932,7926,7914,7893,7914,7933,7905,7932,7850,7879,7917,7911,...,7714,7689,7719,7735,7706,7710,7738,7729,7737,7678,7671,7596,7654,7656,7662,7648,7649,7648,7671,7617,7620,7600,7639,7555,7572,7581,7541,7548,7553,7501,7479,7517,7470,7466,7481,7443,7475,7489,7534,7430
世帯人員(人),3.32,3.32,3.32,3.32,3.31,3.31,3.31,3.31,3.31,3.3,3.3,3.29,3.29,3.29,3.29,3.29,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.28,3.26,3.25,3.25,3.25,3.24,3.24,3.24,3.24,3.24,3.24,3.24,3.23,3.24,3.23,3.23,3.23,...,2.98,2.98,2.97,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.97,2.97,2.97,2.97,2.97,2.98,2.98,2.97,2.98,2.98,2.98,2.98,2.98,2.97,2.97,2.97,2.97,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96,2.96
18歳未満人員(人),0.74,0.75,0.75,0.75,0.75,0.74,0.74,0.74,0.74,0.73,0.72,0.73,0.72,0.72,0.72,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.71,0.7,0.69,0.7,0.69,0.69,0.7,0.7,0.7,0.7,0.71,0.71,0.7,0.69,0.69,0.69,0.68,0.67,...,0.54,0.54,0.54,0.55,0.55,0.55,0.54,0.55,0.55,0.55,0.55,0.55,0.55,0.55,0.54,0.55,0.55,0.56,0.56,0.56,0.57,0.58,0.59,0.58,0.59,0.59,0.59,0.58,0.57,0.57,0.57,0.58,0.58,0.58,0.58,0.58,0.58,0.57,0.58,0.58
65歳以上人員(人),0.52,0.53,0.53,0.52,0.52,0.53,0.54,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.57,0.58,0.57,0.56,0.57,0.57,0.57,0.56,0.55,0.57,0.55,0.57,0.58,0.58,0.58,0.58,0.58,0.59,0.58,0.58,0.58,0.58,0.58,0.59,0.59,0.59,...,0.85,0.86,0.87,0.86,0.86,0.86,0.86,0.85,0.85,0.85,0.85,0.84,0.84,0.84,0.84,0.84,0.84,0.83,0.82,0.83,0.83,0.82,0.82,0.81,0.82,0.83,0.83,0.83,0.83,0.84,0.83,0.82,0.82,0.82,0.82,0.82,0.83,0.86,0.86,0.85


In [14]:
df1 = df1.T
df1

,世帯数分布(抽出率調整),集計世帯数,世帯人員(人),18歳未満人員(人),65歳以上人員(人),うち無職者人員(人),有業人員(人),世帯主の年齢(歳),持家率(％),家賃・地代を支払っている世帯の割合(％),消費支出,食料,穀類,米,パン,麺類,他の穀類,魚介類,生鮮魚介,塩干魚介,魚肉練製品,他の魚介加工品,肉類,生鮮肉,加工肉,乳卵類,牛乳,乳製品,卵,野菜・海藻,生鮮野菜,乾物・海藻,大豆加工品,他の野菜・海藻加工品,果物,生鮮果物,果物加工品,油脂・調味料,油脂,調味料,...,被服及び履物(財・サービス),半耐久財(被服及び履物),サービス(被服及び履物),保健医療(財・サービス),耐久財(保健医療),半耐久財(保健医療),非耐久財(保健医療),サービス(保健医療),交通・通信(財・サービス),耐久財(交通・通信),半耐久財(交通・通信),非耐久財(交通・通信),サービス(交通・通信),教育(財・サービス),非耐久財(教育),サービス(教育),教養娯楽(財・サービス),耐久財(教養娯楽),半耐久財(教養娯楽),非耐久財(教養娯楽),サービス(教養娯楽),諸雑費(財・サービス),耐久財(諸雑費),半耐久財(諸雑費),非耐久財(諸雑費),サービス(諸雑費),調整集計世帯数,教養娯楽関係費,耐久財(教養娯楽関係費),他の教養娯楽関係費,読書,聴視・観覧,旅行,スポーツ,月謝,会費・つきあい費,その他の教養娯楽,消費支出(再掲),基礎的支出,選択的支出
2000-01-31,10000,7887,3.32,0.74,0.52,0.41,1.51,52.4,76,22.1,309621,73580,6100,2338,2127,1325,310,8525,5479,1370,819,858,6229,5144,1085,3390,1590,1114,686,8029,4702,741,1250,1336,2866,2660,205,2897,264,2633,...,19602,18712,891,10941,1123,422,3410,5986,31386,4435,1025,3876,22049,12532,281,12251,31681,4163,4624,6818,16075,19157,797,2064,4047,12249,736445,34008,3912,30096,4304,2049,8876,3001,3241,1561,7063,309621,184372,125248
2000-02-29,10000,7942,3.32,0.75,0.53,0.41,1.51,52.6,76.3,22,290663,73309,6915,2919,2329,1359,308,8627,5475,1500,802,850,6449,5242,1208,3650,1678,1200,772,8570,5102,744,1298,1427,3168,3001,167,3111,272,2838,...,14400,13571,830,11789,853,443,3597,6896,31041,5455,887,3856,20843,14481,220,14260,29737,3026,3937,6892,15882,18800,731,2473,4291,11306,735391,32103,2920,29183,4586,2199,9058,2200,3329,1441,6370,290663,170045,120618
2000-03-31,10000,7934,3.32,0.75,0.53,0.41,1.51,52.7,76.2,22.6,335341,79726,7496,3226,2492,1482,296,9268,5880,1613,794,981,6483,5170,1314,3940,1835,1300,804,9268,5630,841,1311,1487,3270,3111,159,3262,308,2954,...,18366,17174,1192,11767,1043,368,3891,6464,39097,9004,1542,4206,24345,17709,1100,16609,36010,4235,5050,8355,18370,22685,1006,2816,4896,13968,735404,39415,4248,35166,4858,1977,12249,3022,3497,1495,8068,335341,186202,149139
2000-04-30,10000,7922,3.32,0.75,0.52,0.41,1.52,52.6,75.8,22.2,335276,77344,7470,3346,2441,1458,225,8736,5438,1571,761,966,6485,5082,1404,3803,1769,1264,770,9268,5762,828,1262,1415,2999,2848,152,3149,315,2834,...,18028,16141,1887,11251,1000,286,3340,6624,41356,10981,1563,4087,24725,24049,926,23124,34146,2890,4777,8519,17960,20400,580,2216,4528,13076,734606,37458,3109,34349,4552,2470,10432,3129,3768,1888,8109,335276,173441,161835
2000-05-31,10000,7928,3.31,0.75,0.52,0.41,1.53,52.7,76.1,22.6,308566,81415,7447,3264,2458,1523,203,8857,5554,1586,756,961,6655,5168,1487,3899,1877,1258,763,9533,5983,767,1314,1469,3148,2985,162,3307,321,2985,...,18712,16753,1960,11335,965,513,3370,6487,36089,6613,1158,4467,23852,11512,427,11084,33824,2517,3666,9032,18608,19449,452,2321,4698,11979,733789,38525,3354,35171,4516,2202,11746,2946,3256,1683,8822,308566,177153,131413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-31,10000,7443,2.96,0.58,0.82,0.65,1.34,59.3,85.6,13.1,287173,76011,5666,1402,2515,1335,414,5723,3350,990,635,748,7162,5913,1250,3661,1158,1766,737,7832,5102,659,1055,1016,2800,2478,322,3345,339,3006,...,12184,11578,606,13880,979,235,4986,7681,38918,5004,2002,4981,26931,8719,66,8653,28174,2870,3561,5140,16602,25060,518,2542,4929,17071,655791,30567,2995,27572,3187,2590,7299,2839,2648,770,8239,287173,167873,119301
2020-02-29,10000,7475,2.96,0.58,0.83,0.66,1.33,59.3,85.3,13.4,271735,75469,6266,1779,2625,1442,421,5548,3141,991,627,789,7261,5952,1309,3846,1176,1891,779,8081,5192,719,1108,1062,3009,2721,288,3603,366,3237,...,8324,7800,524,14236,834,414,5468,7520,42390,10988,2553,4991,23858,8392,127,8265,25597,1736,3066,5088,15707,22372,331,1679,5189,15173,655490,27381,1696,25684,3184,3149,5922,2436,2614,705,7674,271735,157888,113847
2020-03-31,10000,7489,2.96,0.57,0.86,0

In [15]:
df1.to_csv("./data/today_data.csv")

## データの準備ができました。
- 次に可視化します
- 可視化に使うのは[plotly.express](https://plotly.com/python/plotly-express/)です
- plotly.expressの特徴はインタラクティブなグラフを簡単に作れることです


In [ ]:
# 世帯人員（人）の2000年からの推移の線グラフを作成する

px.line(df1, x=df1.index, y="世帯人員(人)", title="日本の世帯人員数")

In [ ]:
px.line(df1, x=df1.index, y=["18歳未満人員(人)", "65歳以上人員(人)"], title="家庭内の若者数と老人数")

In [ ]:
px.line(df1, x=df1.index, y=["米", "パン"])

## Pandasでのひと手間でもっとインサイトの得られるグラフも
- 季節性を見るために、月をx軸に持ってきて見る

In [16]:
df2 = df1.copy()
df2["year"] = df1.index.year
df2["month"] = df1.index.month

In [17]:
df2.head()

,世帯数分布(抽出率調整),集計世帯数,世帯人員(人),18歳未満人員(人),65歳以上人員(人),うち無職者人員(人),有業人員(人),世帯主の年齢(歳),持家率(％),家賃・地代を支払っている世帯の割合(％),消費支出,食料,穀類,米,パン,麺類,他の穀類,魚介類,生鮮魚介,塩干魚介,魚肉練製品,他の魚介加工品,肉類,生鮮肉,加工肉,乳卵類,牛乳,乳製品,卵,野菜・海藻,生鮮野菜,乾物・海藻,大豆加工品,他の野菜・海藻加工品,果物,生鮮果物,果物加工品,油脂・調味料,油脂,調味料,...,サービス(被服及び履物),保健医療(財・サービス),耐久財(保健医療),半耐久財(保健医療),非耐久財(保健医療),サービス(保健医療),交通・通信(財・サービス),耐久財(交通・通信),半耐久財(交通・通信),非耐久財(交通・通信),サービス(交通・通信),教育(財・サービス),非耐久財(教育),サービス(教育),教養娯楽(財・サービス),耐久財(教養娯楽),半耐久財(教養娯楽),非耐久財(教養娯楽),サービス(教養娯楽),諸雑費(財・サービス),耐久財(諸雑費),半耐久財(諸雑費),非耐久財(諸雑費),サービス(諸雑費),調整集計世帯数,教養娯楽関係費,耐久財(教養娯楽関係費),他の教養娯楽関係費,読書,聴視・観覧,旅行,スポーツ,月謝,会費・つきあい費,その他の教養娯楽,消費支出(再掲),基礎的支出,選択的支出,year,month
2000-01-31,10000,7887,3.32,0.74,0.52,0.41,1.51,52.4,76,22.1,309621,73580,6100,2338,2127,1325,310,8525,5479,1370,819,858,6229,5144,1085,3390,1590,1114,686,8029,4702,741,1250,1336,2866,2660,205,2897,264,2633,...,891,10941,1123,422,3410,5986,31386,4435,1025,3876,22049,12532,281,12251,31681,4163,4624,6818,16075,19157,797,2064,4047,12249,736445,34008,3912,30096,4304,2049,8876,3001,3241,1561,7063,309621,184372,125248,2000,1
2000-02-29,10000,7942,3.32,0.75,0.53,0.41,1.51,52.6,76.3,22,290663,73309,6915,2919,2329,1359,308,8627,5475,1500,802,850,6449,5242,1208,3650,1678,1200,772,8570,5102,744,1298,1427,3168,3001,167,3111,272,2838,...,830,11789,853,443,3597,6896,31041,5455,887,3856,20843,14481,220,14260,29737,3026,3937,6892,15882,18800,731,2473,4291,11306,735391,32103,2920,29183,4586,2199,9058,2200,3329,1441,6370,290663,170045,120618,2000,2
2000-03-31,10000,7934,3.32,0.75,0.53,0.41,1.51,52.7,76.2,22.6,335341,79726,7496,3226,2492,1482,296,9268,5880,1613,794,981,6483,5170,1314,3940,1835,1300,804,9268,5630,841,1311,1487,3270,3111,159,3262,308,2954,...,1192,11767,1043,368,3891,6464,39097,9004,1542,4206,24345,17709,1100,16609,36010,4235,5050,8355,18370,22685,1006,2816,4896,13968,735404,39415,4248,35166,4858,1977,12249,3022,3497,1495,8068,335341,186202,149139,2000,3
2000-04-30,10000,7922,3.32,0.75,0.52,0.41,1.52,52.6,75.8,22.2,335276,77344,7470,3346,2441,1458,225,8736,5438,1571,761,966,6485,5082,1404,3803,1769,1264,770,9268,5762,828,1262,1415,2999,2848,152,3149,315,2834,...,1887,11251,1000,286,3340,6624,41356,10981,1563,4087,24725,24049,926,23124,34146,2890,4777,8519,17960,20400,580,2216,4528,13076,734606,37458,3109,34349,4552,2470,10432,3129,3768,1888,8109,335276,173441,161835,2000,4
2000-05-31,10000,7928,3.31,0.75,0.52,0.41,1.53,52.7,76.1,22.6,308566,81415,7447,3264,2458,1523,203,8857,5554,1586,756,961,6655,5168,1487,3899,1877,1258,763,9533,5983,767,1314,1469,3148,2985,162,3307,321,2985,...,1960,11335,965,513,3370,6487,36089,6613,1158,4467,23852,11512,427,11084,33824,2517,3666,9032,18608,19449,452,2321,4698,11979,733789,38525,3354,35171,4516,2202,11746,2946,3256,1683,8822,308566,177153,131413,2000,5


In [18]:
px.line(df2, x="month", y="パン", color="year")

In [19]:
df2_2010 = df2.query("`year`> 2010")
px.line(df2_2010, x="month", y="米", color="year")

In [20]:
df3 = df1.copy()
df3 = df3.loc[:, "食料": "その他の教養娯楽"]
df3 = df3.drop("調整集計世帯数", axis=1)
df3["date"] = df3.index
df3_melt = df3.melt(id_vars="date")
col_names = ["date", "variable", "value"]
df3_melt.columns = col_names
df3_melt.head()

,date,variable,value
0,2000-01-31,食料,73580
1,2000-02-29,食料,73309
2,2000-03-31,食料,79726
3,2000-04-30,食料,77344
4,2000-05-31,食料,81415


In [21]:
df3_selected = df3_melt.query("`date` == '2020-05-31'")

px.treemap(df3_selected, path=["variable"], values="value")

## Dashを使う
- ここまでだとこれまでのツールと変わらない
- 見たいデータがないんだけどみたいなことを言わさないツール作成
- ここではJupyter上でDashを使うJupyterDashを用いる

### まずは読み込んだデータを使って色々見れるツールを作成します。
- ドロップダウンで選択したらその項目が線グラフで見れる


In [44]:
app = JupyterDash(__name__)

app.layout = html.Div([
                       
        dcc.Dropdown(id='my_dropdown',
                     options=[{"value": col, "label": col} for col in df1.columns],
                     value=[df1.columns[10]],
                     multi=True
                     ),
        dcc.Graph(id="my_graph")
])

@app.callback(Output('my_graph', 'figure'), [Input('my_dropdown', 'value')])
def update_graph(selected_value):
  return px.line(df1, x=df1.index, y=selected_value)

app.run_server(mode='inline')

<IPython.core.display.Javascript object>

## 仕事などでよくありそうな
- エクセルファイルを作ってアップしたらそのまま動きそうなやつ
- 残念ながらコラボ上では動作しませんが・・・
- アプリがヘロク上で動きます
  - https://okodukai.herokuapp.com/
  - 記事を3時間くらいかけて書いたが全くバズらず・・・
  - 憐みのLGTMをください　[Qiita記事](https://qiita.com/OgawaHideyuki/items/f0d647c5b284b338996b)

- コードは[github](https://github.com/mazarimono/okodukai_upload)上にあります

- 試す前に、データを整然データで入れてあげる必要があるので前処理します

In [41]:
df = pd.read_csv("/content/data/today_data.csv", index_col=0)
df['date'] = df.index
df = df.reset_index(drop=True)
df_melt = df.melt(id_vars="date")

In [42]:
df_melt.head()

,date,variable,value
0,2000-01-31,世帯数分布(抽出率調整),10000.0
1,2000-02-29,世帯数分布(抽出率調整),10000.0
2,2000-03-31,世帯数分布(抽出率調整),10000.0
3,2000-04-30,世帯数分布(抽出率調整),10000.0
4,2000-05-31,世帯数分布(抽出率調整),10000.0


In [43]:
df_melt.to_csv("./data/today_melt.csv")

In [46]:
!git config --global user.email "mazarimono@gmail.com"
!git config --global user.name "mazarimono"
